In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/Urdu/urdu_train.csv')
df_val = pd.read_csv("/content/drive/MyDrive/Urdu/urdu_val.csv")

In [3]:
df_train.head()

,text,label
0,نئی دہلی 4 دسمبر (سیاست ڈاٹ کام ) ہندوستانی کر...,R
1,کچے دودھ سے خطرناک جین پھیل سکتے ہیں، تحقیق کی...,R
2,لندن(سپورٹس ڈیسک)ای سی بی کے نیشنل سلیکٹر ایڈ...,R
3,دنیا کی پہلی مصنوعی’ ’تھری ڈی‘‘ آنکھ ہانگ کان...,R
4,ممبئی موٹاپے میں مبتلا ہونے والے بالی ووڈ اد...,F


In [4]:
print(df_train.shape)

df_val.label.value_counts()

(1038, 2)


R    150
F    112
Name: label, dtype: int64

In [5]:
from sklearn.metrics import confusion_matrix
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report


from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [6]:
df_train_Tweet = pd.DataFrame(df_train.text)
df_val_Tweet = pd.DataFrame(df_val.text)

df_train_Label = pd.DataFrame(df_train.label)
df_val_Label = pd.DataFrame(df_val.label)


In [7]:
print(df_train_Tweet.shape,df_val_Label.shape)

(1038, 1) (262, 1)


In [8]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))

def preprocessing(document):
        document = regex.sub(' ', document)
        document = re.sub(r'[0-9]', '', document)
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        return document
    


In [9]:
corpus_train = df_train_Tweet.text.apply(preprocessing)
corpus_val = df_val_Tweet.text.apply(preprocessing)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv = TfidfVectorizer(ngram_range=(1,5))
cv = TfidfVectorizer(max_features=17000)
X_train = cv.fit_transform(corpus_train).toarray()
X_test = cv.transform(corpus_val).toarray()
X_train.shape

(1038, 17000)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler,MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## LOGISTIC REGRESSION

In [12]:
#training - Logistic regression
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs',verbose=3,max_iter=250,penalty='l2',C=1,n_jobs=4)
classifier.fit(X_train , np.ravel(df_train_Label))

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:    7.6s finished


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=250,
                   multi_class='multinomial', n_jobs=4, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=3,
                   warm_start=False)

In [13]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_val_Label , y_pred)
accuracy

0.7519083969465649

In [14]:
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.84      0.52      0.64       112
           R       0.72      0.93      0.81       150

    accuracy                           0.75       262
   macro avg       0.78      0.72      0.73       262
weighted avg       0.77      0.75      0.74       262



## RANDOM FOREST

In [15]:
# Fitting Random Forest Classification to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'gini', random_state = 0 , max_depth=100)
classifier.fit(X_train , np.ravel(df_train_Label))

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_val_Label , y_pred)
accuracy

0.6412213740458015

In [17]:
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.64      0.38      0.47       112
           R       0.64      0.84      0.73       150

    accuracy                           0.64       262
   macro avg       0.64      0.61      0.60       262
weighted avg       0.64      0.64      0.62       262



## NAIVE BAYES

In [18]:
#Naive bayes classification
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB(alpha = 1 , fit_prior=True, class_prior=None)
classifier.fit(X_train , np.ravel(df_train_Label))

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [19]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_val_Label , y_pred)
accuracy

0.7213740458015268

In [20]:
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.81      0.46      0.58       112
           R       0.69      0.92      0.79       150

    accuracy                           0.72       262
   macro avg       0.75      0.69      0.69       262
weighted avg       0.74      0.72      0.70       262



## XGBOOST

In [21]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
classifier = XGBClassifier()
classifier.fit(X_train , np.ravel(df_train_Label))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [23]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_val_Label , y_pred)
accuracy

0.7175572519083969

In [24]:
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       0.76      0.49      0.60       112
           R       0.70      0.89      0.78       150

    accuracy                           0.72       262
   macro avg       0.73      0.69      0.69       262
weighted avg       0.73      0.72      0.70       262



## SVM

In [25]:
from sklearn.svm import SVC
classifier=SVC(random_state=123)
classifier.fit(X_train , np.ravel(df_train_Label))

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)

In [26]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(df_val_Label , y_pred)
accuracy

0.5916030534351145

In [27]:
print(classification_report(df_val_Label,y_pred))

              precision    recall  f1-score   support

           F       1.00      0.04      0.09       112
           R       0.58      1.00      0.74       150

    accuracy                           0.59       262
   macro avg       0.79      0.52      0.41       262
weighted avg       0.76      0.59      0.46       262

